In [1]:
import datetime as dt
import meteomatics.api as api
import pandas as pd
import numpy as np

# Meteomatics api call

The following code can be used to call the meteomatics api.

### WeatherCall function
The WeatherCall function calls the Metomatics api and returns a dataframe with the results of the call.

**Notes on the WeatherCall function:**
* The Metomatics api returns time stamps for averages right bounded (the preceding 24h are taken into account) )in the API because of this the dates are shifted by one day. That means that the following value: 
      validdate;t_mean_2m_24h:F 
      2021-05-06T00:00:00Z;72.4
      represents the 24h hour mean temperature of May 5.
* WeatherCall fixes the date shift for each row returned in a new column called 'date'. The original column with the shifted dates is droped from the dataframe that is returned






 

In [2]:
def WeatherCall(location,parameters,time):
    
    ###Credentials:
    username = 'smu_student'
    password = 'KExx3Nhs768c'
    
    #unpack time
    startdate,enddate,interval = time
    
    #empty results dataframe will be populated by query results
    results = pd.DataFrame()

    #for loop that loops through a list of latitude and longitude locations 
    for x in location:
        try:
            ts = api.query_time_series(x, startdate, enddate, interval,parameters, username, password) #query api
            ts_df = ts.reset_index() #lat, lon, and validdate come back as a multi Index this makes them columns
            results = results.append(ts_df) #append the results dataframe with query results
            results['date'] = results.validdate - dt.timedelta(days=1) #daily results are off by a day


        except Exception as e:
            print("Failed, the exception is {}".format(e)) 
        
    results = results.drop(columns=['validdate'])
    return results    

### Import mgrs location csv and format to work with WeatherCall function

The meteomatics api can only be queried for 10 locations at a time. To return results for all 385 mgrs locations the latitude and longitude have to be combined into individual records (tuples) and then sent through the api ten at a time. The following code will pull down the mgrs_final.csv file from our git convert the lat and long to tuples and put them in a list. The code will then split the orginal list into a list of lists each of the smaller lists containing ten locations except for the last list which only contains five locations. The code returns 'mgrs_locations' that can be passed into the 'WeatherCall' function to get results for all 385 locations at once. 

**<font color=red>Notes on location READ ME: </font>**
   * When 'mgrs_locations' list is sent through the 'WeatherCall' function 39 calls will be made to the api! One call = list of 10 locations

  * When 'mgrs_test' list is sent through the 'WeatherCall funciton 2 calls will be made to the api. Use this for testing until you're ready to get data for all lcoations.

In [3]:
from itertools import islice

#Import mgrs locations
mgrs = pd.read_csv('https://raw.githubusercontent.com/jbalson0808/fires/main/mgrs_final.csv') #set location of mgrs_final

#Select lat and longitude columns
mgrs = mgrs[['lat_10km','lon_10km']]

#Convert mgrs Lat and lon into a list of tuples 
records = mgrs.to_records(index=False)

#convert 
mgrs_list = list(records)

#split list into 39 smaller lists 
length_to_split = [10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,6]

inputt = iter(mgrs_list)

# All mgrs locations ################
mgrs_locations = [list(islice(inputt,elem)) for elem in length_to_split]


# mgrs_test this limits the locations to twenty and only makes 2 calls to the api useful in testing#######
mgrs_test = mgrs_locations[:2]

#print(mgrs_locations[:2]

In [4]:
mgrs_locations[:2]

[[(33.70526732, -118.079125),
  (33.79544412, -118.0802553),
  (33.88561957, -118.0813907),
  (33.97579365, -118.0825311),
  (34.06596638, -118.0836766),
  (34.15613776, -118.0848272),
  (34.24630777, -118.085983),
  (33.61598164, -117.970214),
  (33.70616282, -117.9712268),
  (33.79634266, -117.9722441)],
 [(33.88652115, -117.973266),
  (33.97669829, -117.9742925),
  (34.06687408, -117.9753235),
  (34.15704851, -117.9763591),
  (34.24722159, -117.9773993),
  (33.6167802, -117.8624237),
  (33.70696409, -117.8633241),
  (33.79714665, -117.8642284),
  (33.88732786, -117.8651368),
  (33.97750773, -117.8660493)]]

### Set Start date and enddate of call
Use the code below to set the start and end dates for the weather data you want to call. The start date, end date, and interval are packed into a variable called 'time' that can be passed to the WeatherCall function. 

Notes on start and end date: 
* the Meteomatics api uses right bounded dates for averages when entering start date and end date you need to shift the dates by one day. Example, if you want to get the daily mean (00:00 until 00:00 UTC) temperatures between May 5 and 10, you should specify the following:
        startdate_ts = dt.datetime(2021,5,6)
        enddate_ts = dt.datetime(2021,5,11)
        interval_ts = dt.timedelta(hours=24) 
        
* The code below shifts the start and end dates you enter by one day to make it easier to use. Just enter the actual dates you want and the function will do the rest


In [5]:
# month_s = 1    #start date month
# day_s = 1      #start date day
# year_s = 2000  #start date year

# month_e = 12    #end date month
# day_e = 31     #end date day
# year_e = 2010  #end date year

month_s = 1   #start date month
day_s = 1      #start date day
year_s = 2013  #start date year ##!!3 years before 2016 for mgrs_fires_past3yrs rolling average

month_e = 12  #end date month
day_e = 31     #end date day
year_e = 2020  #end date year

########### function to set WeatherCall time ##########################
def t(year_s,month_s,day_s,year_e,month_e,day_e):
    
    #start date +1day
    startdate = dt.datetime(year_s,month_s,day_s) + dt.timedelta(days=1) 
    
    #end date +1day
    enddate = dt.datetime(year_e,month_e,day_e) + dt.timedelta(days=1) 
    
    #interval hours = 24 for daily data
    interval = dt.timedelta(hours=24)
    
    time = startdate,enddate,interval
    
    return time
########################################################################

time = t(year_s,month_s,day_s,year_e,month_e,day_e)

### Set parameters you want to return

Use the code below to set the values of the parameters you want the api to return.Temperature and other parameters available that can be queried for 24 h periods, like dew point and relative humidity also 24 h precipitation sums can be queried. 
* Parameters that can be called are found in the Metomatics Documentation here: https://www.meteomatics.com/en/api/available-parameters/

In [6]:
parameters = ['t_mean_2m_24h:F', 't_min_2m_24h:F','t_max_2m_24h:F','precip_24h:mm','sunshine_duration_24h:min','drought_index:idx','soil_moisture_index_-15cm:idx','soil_type:idx','wind_speed_2m:mph','elevation:m']

# Call Meteomatics API with WeatherCall

**Run the code below to call the Metomatics API after setting location, time, and parameters above.**


In [7]:
#Call API
# results = WeatherCall(mgrs_test,parameters,time)
results = WeatherCall(mgrs_locations,parameters,time)

#View Results
results.head()

,lat,lon,t_mean_2m_24h:F,t_min_2m_24h:F,t_max_2m_24h:F,precip_24h:mm,sunshine_duration_24h:min,drought_index:idx,soil_moisture_index_-15cm:idx,soil_type:idx,wind_speed_2m:mph,elevation:m,date
0,33.705267,-118.079125,54.0,43.3,64.6,0.00,589.8,-1.0,NaN,NaN,4.1,0.0,2016-01-01 00:00:00+00:00
1,33.705267,-118.079125,54.8,47.9,62.9,0.00,431.7,-2.0,NaN,NaN,4.2,0.0,2016-01-02 00:00:00+00:00
2,33.705267,-118.079125,54.8,47.1,62.3,0.00,584.2,-3.0,NaN,NaN,8.0,0.0,2016-01-03 00:00:00+00:00
3,33.705267,-118.079125,57.4,52.7,63.5,1.61,242.0,-2.0,NaN,NaN,6.5,0.0,2016-01-04 00:00:00+00:00
4,33.705267,-118.079125,57.6,54.2,62.8,21.89,129.9,-1.0,NaN,NaN,8.7,0.0,2016-01-05 00:00:00+00:00


In [8]:
results

,lat,lon,t_mean_2m_24h:F,t_min_2m_24h:F,t_max_2m_24h:F,precip_24h:mm,sunshine_duration_24h:min,drought_index:idx,soil_moisture_index_-15cm:idx,soil_type:idx,wind_speed_2m:mph,elevation:m,date
0,33.705267,-118.079125,54.0,43.3,64.6,0.00,589.8,-1.0,NaN,NaN,4.1,0.0,2016-01-01 00:00:00+00:00
1,33.705267,-118.079125,54.8,47.9,62.9,0.00,431.7,-2.0,NaN,NaN,4.2,0.0,2016-01-02 00:00:00+00:00
2,33.705267,-118.079125,54.8,47.1,62.3,0.00,584.2,-3.0,NaN,NaN,8.0,0.0,2016-01-03 00:00:00+00:00
3,33.705267,-118.079125,57.4,52.7,63.5,1.61,242.0,-2.0,NaN,NaN,6.5,0.0,2016-01-04 00:00:00+00:00
4,33.705267,-118.079125,57.6,54.2,62.8,21.89,129.9,-1.0,NaN,NaN,8.7,0.0,2016-01-05 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9130,33.167642,-117.536254,57.6,55.4,60.2,0.00,187.9,-4.0,NaN,NaN,4.9,0.0,2020-12-27 00:00:00+00:00
9131,33.167642,-117.536254,56.8,54.2,58.8,7.38,283.2,-4.0,NaN,NaN,11.4,0.0,2020-12-28 00:00:00+00:00
9132,33.167642,-117.536254,54.8,52.4,60.6,10.16,514.3,-4.0,NaN,NaN,12.2,0.0,2020-12-29 00:00:00+00:00
9133,33.167642,-117.536254,57.0,52.9,60.8,0.00,599.0,-4.0,NaN,NaN,5.2,0.0,2020-12-30 00:00:00+00:00


In [12]:
results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 703395 entries, 0 to 9134
Data columns (total 13 columns):
 #   Column                         Non-Null Count   Dtype              
---  ------                         --------------   -----              
 0   lat                            703395 non-null  float64            
 1   lon                            703395 non-null  float64            
 2   t_mean_2m_24h:F                703395 non-null  float64            
 3   t_min_2m_24h:F                 703395 non-null  float64            
 4   t_max_2m_24h:F                 703395 non-null  float64            
 5   precip_24h:mm                  703395 non-null  float64            
 6   sunshine_duration_24h:min      703395 non-null  float64            
 7   drought_index:idx              703395 non-null  float64            
 8   soil_moisture_index_-15cm:idx  626661 non-null  float64            
 9   soil_type:idx                  630315 non-null  float64            
 10  wind_speed

In [13]:
weather = results.copy()
weather.to_csv('2016to2020_Weather_v1.csv')